In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

# Parametry wejściowe – tylko ta jedna stacja
station_id = 731  # Gdańsk, ul. Wyzwolenia
start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 3, 31)

# Funkcja do pobrania listy sensorów tylko dla tej stacji
def get_sensors(station_id):
    url = f"https://api.gios.gov.pl/pjp-api/v1/rest/station/sensors/{station_id}"
    resp = requests.get(url)
    resp.raise_for_status()

    data = resp.json()
    sensors = data.get("Lista stanowisk pomiarowych dla podanej stacji", [])

    if isinstance(sensors, list) and all(isinstance(s, dict) for s in sensors):
        return sensors
    else:
        print("⚠️ Nieprawidłowy format listy sensorów:", sensors)
        return []

# Funkcja do pobrania danych pomiarowych z paginacją
def get_archival_data(sensor_id, date_from_str, date_to_str):
    base_url = f"https://api.gios.gov.pl/pjp-api/v1/rest/archivalData/getDataBySensor/{sensor_id}"
    params = {
        "dateFrom": date_from_str,
        "dateTo": date_to_str,
        "page": 0,
        "size": 100
    }

    all_data = []
    while True:
        try:
            resp = requests.get(base_url, params=params)
            if resp.status_code == 429:
                print("Zbyt wiele zapytań – czekam 10 sekund...")
                time.sleep(2) # TUTAJ OCZEKIWANIE
                continue

            resp.raise_for_status()
            data = resp.json()
            measurements = data.get("Lista archiwalnych wyników pomiarów", [])
            all_data.extend(measurements)

            total_pages = data.get("totalPages", 1)
            if params["page"] >= total_pages - 1:
                break

            params["page"] += 1
            time.sleep(1.2)
        except Exception as e:
            print(f"Błąd przy pobieraniu danych z sensora {sensor_id}: {e}")
            break

    return all_data

# Funkcja do przetwarzania danych na średnie dobowe
def process_hourly_to_daily(data):
    df = pd.DataFrame(data)
    if df.empty:
        return df

    df["Data"] = pd.to_datetime(df["Data"])
    df = df.rename(columns={"Wartość": "value"})
    df = df.dropna(subset=["value"])
    df["date"] = df["Data"].dt.date
    daily_avg = df.groupby("date")["value"].mean().reset_index()
    return daily_avg

# Pobieranie listy sensorów dla stacji 731
print("📡 Pobieranie listy sensorów dla stacji 731 (Gdańsk, ul. Wyzwolenia)...")
sensors = get_sensors(station_id)

result_dfs = []

for sensor in sensors:
    sensor_id = sensor.get("Identyfikator stanowiska")
    param_formula = sensor.get("Wskaźnik - wzór")

    if param_formula not in ["PM10", "PM2.5"]:
        continue  # pomijamy inne sensory

    print(f"🔍 Sensor: {param_formula} (ID: {sensor_id})")

    current_start = start_date
    all_records = []

    while current_start <= end_date:
        segment_end = min(current_start + timedelta(days=365), end_date)
        date_from_str = current_start.strftime("%Y-%m-%d %H:%M")
        date_to_str = segment_end.strftime("%Y-%m-%d %H:%M")

        print(f"  📅 Zakres: {date_from_str} → {date_to_str}")
        data = get_archival_data(sensor_id, date_from_str, date_to_str)
        all_records.extend(data)

        current_start = segment_end + timedelta(days=1)
        time.sleep(1.5)

    daily_df = process_hourly_to_daily(all_records)

    if not daily_df.empty:
        daily_df.rename(columns={"value": param_formula}, inplace=True)
        result_dfs.append(daily_df.set_index("date"))

# Łączenie i zapisywanie danych
if result_dfs:
    final_df = pd.concat(result_dfs, axis=1)
    final_df.sort_index(inplace=True)
    final_df.to_csv("pomiar2025.csv")
    print("✅ Dane zapisane do pliku: pomiar_pm10_pm25_gdansk_ul_wyzwolenia.csv")
else:
    print("⚠️ Brak danych do zapisania.")